In [ ]:
import os
import glob
import time
import pandas as pd

# 🔹 Google Drive Mount
from google.colab import drive
drive.mount('/content/drive')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# =============== 0. ISBN 정규화 함수 ===============
def normalize_isbn13(value) -> str:
    if pd.isna(value):
        return ""
    if isinstance(value, (int, float)):
        try:
            return f"{int(value):013d}"
        except:
            pass

    s = str(value).strip()
    if s.endswith(".0"):
        s = s[:-2]

    s = s.replace("-", "").replace(" ", "")
    digits = "".join(ch for ch in s if ch.isdigit())

    return digits


# =============== 1. Selenium 드라이버 세팅 (Colab용) ===============
def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")   # Colab은 headless 권장
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option("excludeSwitches", ["enable-logging"])

    driver = webdriver.Chrome(options=options)
    return driver


# =============== 2. YES24에서 ISBN으로 ‘목차’ 가져오기 ===============
def get_yes24_contents(driver, isbn: str, wait: WebDriverWait) -> str | None:
    """
    yes24 메인 → 검색 → 도서 상세 → 목차(infoset_contents) 텍스트 반환
    """
    try:
        driver.get("https://www.yes24.com")
        time.sleep(1)

        # 검색창
        search_input = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="yesSForm"]//input[@type="text"]')
            )
        )
        search_input.clear()
        search_input.send_keys(isbn)

        # 검색 버튼
        search_button = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, '//*[@id="yesSForm"]/fieldset/span[2]/button')
            )
        )
        search_button.click()
        time.sleep(1.5)

        # 첫 번째 도서 클릭
        try:
            first_book = wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//*[@id="yesSchList"]/li[1]/div/div[1]/div[1]/span[1]/span[1]/a[1]')
                )
            )
        except TimeoutException:
            print(f"[WARN] ISBN {isbn} : 검색 결과 없음")
            return None

        first_book.click()
        time.sleep(1.5)

        # 🔥 핵심: 책소개 → 목차로 변경
        # YES24 '목차' 영역: id="infoset_contents"
        try:
            contents_elem = wait.until(
                EC.visibility_of_element_located(
                    (By.ID, "infoset_contents")
                )
            )
            contents_text = contents_elem.text.strip()
            if not contents_text:
                print(f"[WARN] ISBN {isbn} : 목차 영역 비어 있음")
                return None
            return contents_text

        except TimeoutException:
            print(f"[WARN] ISBN {isbn} : 목차(infoset_contents) 영역 없음")
            return None

    except Exception as e:
        print(f"[ERROR] ISBN {isbn} 오류: {e}")
        return None


# =============== 3. Google Drive 폴더 안 엑셀 파일 처리 ===============

# 🔹 Google Drive 경로로 변경!
base_dir = "/content/drive/MyDrive/2025/2-2. 빅데이터와 지식관리시스템/Aladin_crawling"         # 원본 엑셀 폴더
output_dir = "/content/drive/MyDrive/2025/2-2. 빅데이터와 지식관리시스템/Aladin_crawling_content"   # 결과 저장 폴더
os.makedirs(output_dir, exist_ok=True)

# 엑셀 파일 목록
excel_files = [
    f for f in glob.glob(os.path.join(base_dir, "*.xlsx"))
    if not os.path.basename(f).startswith("~$")
]

print(f"[INFO] 총 {len(excel_files)}개의 엑셀 파일 발견")

# Selenium 드라이버 생성
driver = create_driver()
wait = WebDriverWait(driver, 10)

try:
    for file_path in excel_files:
        print("\n=====================================================")
        print(f"[PROCESS] 파일 처리 시작: {file_path}")

        df = pd.read_excel(file_path, dtype={"ISBN13": str})
        df_out = df.copy()

        # 🔥 책소개 → 목차 컬럼으로 변경
        df_out["목차"] = None

        for idx, row in df_out.iterrows():
            raw_isbn = row.get("ISBN13")
            isbn = normalize_isbn13(raw_isbn)

            if not isbn:
                print(f"[SKIP] idx {idx}: ISBN 없음")
                continue

            print(f"[INFO] ({idx+1}/{len(df_out)}) ISBN={isbn}")

            contents = get_yes24_contents(driver, isbn, wait)

            if contents:
                df_out.at[idx, "목차"] = contents
                print(f"  [DONE] 목차 저장 (길이 {len(contents)})")
            else:
                df_out.at[idx, "목차"] = ""
                print("  [DONE] 목차 없음 → 빈 문자열 저장")

            time.sleep(1.0)

        filename = os.path.basename(file_path)
        name_no_ext = os.path.splitext(filename)[0]
        output_file = os.path.join(output_dir, f"{name_no_ext}_with_contents.xlsx")

        df_out.to_excel(output_file, index=False)
        print(f"[SAVE] 저장 완료 → {output_file}")

finally:
    driver.quit()
    print("\n[INFO] Selenium 종료 완료")

print("\n[ALL DONE] 전체 파일 처리 완료 🎉")

MessageError: Error: credential propagation was unsuccessful

In [ ]:
!apt-get update
!apt-get install -y chromium-browser
!apt-get install -y chromium-chromedriver
!pip install selenium

Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
!which chromium-browser
!which chromedriver

/usr/bin/chromium-browser
/usr/bin/chromedriver


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

def create_driver():
    chrome_options = webdriver.ChromeOptions()

    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--remote-debugging-port=9222")
    chrome_options.binary_location = "/usr/bin/chromium-browser"

    service = Service("/usr/bin/chromedriver")

    return webdriver.Chrome(service=service, options=chrome_options)

# ✔ YES24 목차 영역 ID = infoset_contents
TARGET_ID = "infoset_contents"

def create_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    return webdriver.Chrome(options=options)

def test_single_isbn(isbn):
    driver = create_driver()
    wait = WebDriverWait(driver, 10)

    try:
        print(f"[TEST] ISBN = {isbn}")

        # 1) YES24 접속
        driver.get("https://www.yes24.com")
        time.sleep(1)

        # 2) 검색창 입력
        search_input = wait.until(
            EC.presence_of_element_located(
                (By.XPATH, '//*[@id="yesSForm"]//input[@type="text"]')
            )
        )
        search_input.clear()
        search_input.send_keys(isbn)

        # 3) 검색 버튼 클릭
        search_btn = wait.until(
            EC.element_to_be_clickable(
                (By.XPATH, '//*[@id="yesSForm"]/fieldset/span[2]/button')
            )
        )
        search_btn.click()
        time.sleep(1.5)

        # 4) 첫 번째 도서 클릭
        try:
            first_book = wait.until(
                EC.element_to_be_clickable(
                    (By.XPATH, '//*[@id="yesSchList"]/li[1]/div/div[1]/div[1]/span[1]/span[1]/a[1]')
                )
            )
        except TimeoutException:
            print("[WARN] 검색 결과 없음")
            return None

        first_book.click()
        time.sleep(1.5)

        # 5) 목차 영역 가져오기
        try:
            contents = wait.until(
                EC.visibility_of_element_located(
                    (By.ID, TARGET_ID)
                )
            )
            text = contents.text.strip()
            if text:
                print("\n===== [SUCCESS] 목차 내용 =====")
                print(text[:400], "...\n")  # 너무 길면 앞부분만 출력
                return text
            else:
                print("[WARN] 목차 영역 비어 있음")
                return None
        except TimeoutException:
            print("[ERROR] 목차(infoset_contents) 영역을 찾지 못함")
            return None

    except Exception as e:
        print(f"[ERROR] 실행 중 예외 발생: {e}")

    finally:
        driver.quit()
        print("\n[INFO] 테스트 종료")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
test_single_isbn("9791163034853")

SessionNotCreatedException: Message: session not created: Chrome instance exited. Examine ChromeDriver verbose log to determine the cause.; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#sessionnotcreatedexception
Stacktrace:
#0 0x5c5494d9216a <unknown>
#1 0x5c549480fc4b <unknown>
#2 0x5c5494849d1d <unknown>
#3 0x5c54948456d3 <unknown>
#4 0x5c549489524c <unknown>
#5 0x5c549489496c <unknown>
#6 0x5c5494853c42 <unknown>
#7 0x5c54948548f1 <unknown>
#8 0x5c5494d5af09 <unknown>
#9 0x5c5494d5de4d <unknown>
#10 0x5c5494d43c51 <unknown>
#11 0x5c5494d5ea2b <unknown>
#12 0x5c5494d2aa20 <unknown>
#13 0x5c5494d7fa78 <unknown>
#14 0x5c5494d7fc49 <unknown>
#15 0x5c5494d914c3 <unknown>
#16 0x7f52a0193ac3 <unknown>


In [ ]:
!which chromium-browser

!chromium-browser --version

!chromedriver --version

/usr/bin/chromium-browser

Command '/usr/bin/chromium-browser' requires the chromium snap to be installed.
Please install it with:

snap install chromium


Command '/usr/bin/chromedriver' requires the chromium snap to be installed.
Please install it with:

snap install chromium



In [ ]:
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 MB 16.8 MB/s eta 0:00:00
173.9 MiB [] 0% 264.6s173.9 MiB [] 0% 27.3s173.9 MiB [] 0% 17.6s173.9 MiB [] 0% 8.5s173.9 MiB [] 1% 5.6s173.9 MiB [] 1% 4.3s173.9 MiB [] 2% 3.3s173.9 MiB [] 3% 2.8s173.9 MiB [] 4% 2.6s173.9 MiB [] 5% 2.7s173.9 MiB [] 6% 2.6s173.9 MiB [] 7% 2.5s173.9 MiB [] 8% 2.3s173.9 MiB [] 9% 2.3s173.9 MiB [] 11% 2.1s173.9 MiB [] 12% 2.0s173.9 MiB [] 13% 2.0s173.9 MiB [] 14% 1.9s173.9 MiB [] 15% 1.9s173.9 MiB [] 16% 1.8s173.9 MiB [] 17% 1.8s173.9 MiB [] 18% 1.7s173.9 MiB [] 20% 1.7s173.9 MiB [] 21% 1.6s173.9 MiB [] 22% 1.6s173.9 MiB [] 23% 1.5s173.9 MiB [] 24% 1.5s173.9 MiB [] 25% 1.5s173.9 MiB [] 26% 1.5s173.9 MiB [] 28% 1.5s173.9 MiB [] 29% 1.4s173.9 MiB [] 31% 1.4s173.9 MiB [] 32% 1.3s173.9 MiB [] 33% 1.3s173.9 MiB [] 35% 1.2s173.9 MiB [] 36% 1.2s173.9 MiB [] 38% 1.1s173.9 MiB [] 39% 1.1s173.9 MiB [] 41% 1.0s173.9 MiB [] 43% 1.0s173.9 MiB [] 44% 1.0s173.9 MiB [] 45% 0.9s173.9 MiB [] 46% 0.9s173.9 MiB [] 48% 0.9s173.9 M

In [ ]:
from playwright.sync_api import sync_playwright

def yes24_get_contents(isbn):
    print(f"[TEST] ISBN={isbn} 크롤링 시작")

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True)
        page = browser.new_page()

        # YES24 접속
        page.goto("https://www.yes24.com", timeout=60000)

        # 검색어 입력
        page.fill('input[type="text"]', isbn)
        page.click('#yesSForm fieldset span:nth-child(2) button')

        # 첫 번째 검색 결과 클릭
        try:
            page.wait_for_selector('#yesSchList li:first-child a', timeout=6000)
            page.click('#yesSchList li:first-child a')
        except:
            print("[WARN] 검색결과 없음")
            return None

        # 목차 영역 로드
        try:
            page.wait_for_selector('#infoset_contents', timeout=8000)
            text = page.inner_text('#infoset_contents').strip()
            if text:
                print("\n===== [SUCCESS] 목차 중 일부 =====")
                print(text[:400], "...")
                return text
            else:
                print("[WARN] 목차 비어 있음")
                return None
        except:
            print("[ERROR] 목차 영역 없음")
            return None
        finally:
            browser.close()


# 실행 테스트
yes24_get_contents("9791163034853")


[TEST] ISBN=9791163034853 크롤링 시작


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.